In [1]:
!pip install -q docling docling-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 22.4 MB/s eta 0:00:00
   ━━━━

In [2]:
import time

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import DocumentConverter, PdfFormatOption

2025-05-13 14:38:09.897101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747147090.339099      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747147090.460330      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 1. Khởi tạo Docling converter

In [3]:
# Define pipeline options for PDF processing
pipeline_options = PdfPipelineOptions(
    do_table_structure=True,  # Enable table structure detection
    do_ocr=False,  # Enable OCR
    # full page ocr and language selection
    # ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),  # Use EasyOCR for OCR
    # ocr_options=TesseractOcrOptions(force_full_page_ocr=True, lang=["eng"]),  # Uncomment to use Tesseract for OCR
    # ocr_options = OcrMacOptions(force_full_page_ocr=True, lang=['en-US']),
    table_structure_options=dict(
        do_cell_matching=False,  # Use text cells predicted from table structure model
        mode=TableFormerMode.ACCURATE,  # Use more accurate Tableformer model
    ),
    generate_page_images=False,  # Enable page image generation
    generate_picture_images=False,  # Enable picture image generation
    # images_scale=IMAGE_RESOLUTION_SCALE,  # Set image resolution scale (scale=1 corresponds to a standard 72 DPI image)
)


format_options = {InputFormat.PDF: PdfFormatOption(
    pipeline_options=pipeline_options)}
converter = DocumentConverter(format_options=format_options)


def convert_pdf_to_docling(sources: list[str]):
    start_time = time.time()
    conversions = {source: converter.convert(
        source).document for source in sources}
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f}")
    return conversions

## 2. Các hàm hữu ích

In [4]:
import csv
import re
from io import StringIO

import numpy as np
import pandas as pd


def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess DataFrame to handle common header issues:
    - If column names are numbers, use first row as header
    - If first data row is numbers (0-based or 1-based), remove it
    """
    if df.empty:
        return df

    df_processed = df.copy()

    # Handle case where column names are numbers
    if all(str(col).isdigit() for col in df_processed.columns):
        if not df_processed.empty and df_processed.shape[0] > 0:
            new_cols = df_processed.iloc[0].astype(str).values
            df_processed = df_processed.iloc[1:].reset_index(drop=True)
            df_processed.columns = new_cols

    # Handle case where first data row is a sequence of numbers
    elif not df_processed.empty and df_processed.shape[0] > 0:
        first_row_values = df_processed.iloc[0]
        try:
            numeric_first_row = pd.to_numeric(first_row_values)
            is_0_based_sequence = all(
                numeric_first_row.iloc[i] == i for i in range(len(numeric_first_row))
            )
            is_1_based_sequence = all(
                numeric_first_row.iloc[i] == i + 1
                for i in range(len(numeric_first_row))
            )

            if is_0_based_sequence or is_1_based_sequence:
                df_processed = df_processed.iloc[1:].reset_index(drop=True)
        except (ValueError, TypeError):
            pass

    return df_processed


def solve_non_header_table(df: pd.DataFrame, target_headers: list) -> pd.DataFrame:
    """
    Convert headerless DataFrame by moving current column names to first data row,
    then assign target_headers
    """
    if not isinstance(target_headers, list):
        print("Warning: target_headers is not a list. Returning original DataFrame.")
        return df.copy()

    df_copy = df.copy()

    first_row_data_values = []
    for col_original_name in df_copy.columns:
        if isinstance(col_original_name, str) and col_original_name.startswith(
            "Unnamed:"
        ):
            first_row_data_values.append(np.nan)
        else:
            first_row_data_values.append(str(col_original_name))

    if len(first_row_data_values) != len(target_headers):
        if len(first_row_data_values) > len(target_headers):
            first_row_data_values = first_row_data_values[: len(
                target_headers)]
        else:
            first_row_data_values.extend(
                [np.nan] * (len(target_headers) - len(first_row_data_values))
            )

    new_first_row_df = pd.DataFrame(
        [first_row_data_values], columns=target_headers)

    try:
        df_copy.columns = target_headers
    except ValueError as e:
        print(
            f"Critical error: Cannot assign target_headers. "
            f"DataFrame columns ({len(df_copy.columns)}) don't match target_headers length ({len(target_headers)}). Error: {e}"
        )
        return new_first_row_df

    result_df = pd.concat([new_first_row_df, df_copy], ignore_index=True)

    return result_df.reset_index(drop=True)


def get_column_types(df):
    """Determine data types for each column"""
    types = []
    if df.columns.empty:
        return types

    for col_name in df.columns:
        current_column_object = df[col_name]

        is_truly_all_na = False
        effective_dtype = None

        if isinstance(current_column_object, pd.DataFrame):
            is_truly_all_na = pd.isna(current_column_object).all().all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.iloc[:, 0].dtype
        elif isinstance(current_column_object, pd.Series):
            is_truly_all_na = pd.isna(current_column_object).all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.dtype
        else:
            types.append("error_unexpected_type")
            continue

        if is_truly_all_na:
            types.append("unknown")
        else:
            if effective_dtype is not None:
                types.append(
                    "numeric"
                    if pd.api.types.is_numeric_dtype(effective_dtype)
                    else str(effective_dtype)
                )
            else:
                types.append("unknown_error_in_logic")
    return types


def get_input_df(
    df: pd.DataFrame, n_rows: int = 10, sep: str = "=", max_tokens: int = 15
) -> str:
    """
    Get input DataFrame with sampling and formatting for LLM processing
    """
    df_copy = df.copy()
    df_copy = solve_non_header_table(
        df_copy, [str(x) for x in range(len(df_copy.columns))])

    column_data_types = get_column_types(df_copy)

    for dtype, col_name in zip(column_data_types, df_copy.columns):
        if dtype == "object":
            if isinstance(df_copy[col_name], pd.Series):
                current_series = df_copy[col_name]
                try:
                    df_copy.loc[:, col_name] = current_series.astype(str).apply(
                        lambda x: " ".join(x.split()[:max_tokens])
                    )
                except AttributeError:
                    df_copy.loc[:, col_name] = current_series.apply(
                        lambda x: " ".join(str(x).split()[:max_tokens])
                    )
                pass

    if df_copy.empty:
        return df_copy.to_csv(index=False, header=False, sep=sep)

    if len(df_copy) < n_rows:
        return df_copy.to_csv(index=False, header=False, sep=sep)

    n_edge = n_rows // 3
    n_random = n_rows - (2 * n_edge)

    if n_edge * 2 >= len(df_copy):
        n_edge = len(df_copy) // 2
        n_random = max(0, n_rows - 2 * n_edge)
        if len(df_copy) < n_rows and len(df_copy) > 0:
            n_edge = len(df_copy)
            n_random = 0

    first_rows = df_copy.head(n_edge)
    last_rows = df_copy.tail(n_edge)

    sampled_rows_list = []
    if not first_rows.empty:
        sampled_rows_list.append(first_rows)

    middle_df_start_idx = first_rows.shape[0]
    middle_df_end_idx = len(df_copy) - last_rows.shape[0]

    if middle_df_start_idx < middle_df_end_idx and n_random > 0:
        middle_section = df_copy.iloc[middle_df_start_idx:middle_df_end_idx]
        if not middle_section.empty:
            actual_n_random = min(n_random, len(middle_section))
            if actual_n_random > 0:
                random_rows = middle_section.sample(
                    n=actual_n_random, random_state=42)
                if not random_rows.empty:
                    sampled_rows_list.append(random_rows)

    if not last_rows.empty:
        is_duplicate = False
        temp_concat_check = (
            pd.concat(sampled_rows_list) if sampled_rows_list else pd.DataFrame()
        )
        if not temp_concat_check.empty:
            merged_check = pd.merge(temp_concat_check, last_rows, how="inner")
            if len(merged_check) == len(last_rows) and len(merged_check) == len(
                temp_concat_check
            ):
                is_duplicate = True
                if len(first_rows) + (
                    len(random_rows)
                    if "random_rows" in locals() and not random_rows.empty
                    else 0
                ) == len(last_rows):
                    is_duplicate = True

        if len(sampled_rows_list) == 1 and last_rows.equals(sampled_rows_list[0]):
            is_duplicate = True

        if not is_duplicate:
            sampled_rows_list.append(last_rows)

    if not sampled_rows_list:
        return df_copy.head(0).to_csv(index=False, header=False, sep=sep)

    final_df_to_export = (
        pd.concat(sampled_rows_list).drop_duplicates().reset_index(drop=True)
    )

    return final_df_to_export.to_csv(index=False, header=False, sep=sep)

## 3. Xử lý

Duyệt qua Docling và lấy các tables dưới dạng `pd.DataFrame` và tạo ra `input_prompt` cho LLM

In [5]:
import pandas as pd
from docling_core.types.doc.document import TableItem, TextItem  # Cần import TextItem
from docling_core.types.doc.labels import DocItemLabel


def _get_item_from_cref(docling_document, cref: str):
    try:
        parts = cref.strip('#/').split('/')
        item_type = parts[0]  # 'texts' hoặc 'tables'
        item_index = int(parts[1])
        if item_type == 'texts' and item_index < len(docling_document.texts):
            return docling_document.texts[item_index]
        elif item_type == 'tables' and item_index < len(docling_document.tables):
            return docling_document.tables[item_index]
        else:
            print(f"Warning: Cref '{cref}' không hợp lệ hoặc không tìm thấy.")
            return None
    except (IndexError, ValueError, AttributeError) as e:
        print(f"Error parsing cref '{cref}': {e}")
        return None

In [6]:
import pandas as pd
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem
from docling_core.types.doc.labels import DocItemLabel
from langchain_core.documents import Document


def get_tables_from_docling(docling_document, n_tokens_previous: int = 20) -> tuple[list[pd.DataFrame], str]:
    tables: list[pd.DataFrame] = []
    full_prompt = ""
    table_index = 0

    doc_items = docling_document.body.children if hasattr(
        docling_document, 'body') and hasattr(docling_document.body, 'children') else []

    for i, item_ref in enumerate(doc_items):
        # Kiểm tra xem item hiện tại có phải là một bảng không
        if item_ref.cref.startswith('#/tables/'):
            table_item = _get_item_from_cref(docling_document, item_ref.cref)

            # Chỉ xử lý nếu là TableItem hợp lệ và có label là TABLE
            if isinstance(table_item, TableItem) and table_item.label in [DocItemLabel.TABLE]:

                # --- Lấy ngữ cảnh văn bản phía trước ---
                previous_text_snippet = ""
                if i > 0:  # Đảm bảo không phải là item đầu tiên
                    prev_item_ref = doc_items[i-1]
                    # Kiểm tra xem item trước đó có phải là text không
                    if prev_item_ref.cref.startswith('#/texts/'):
                        prev_text_item = _get_item_from_cref(
                            docling_document, prev_item_ref.cref)
                        if isinstance(prev_text_item, TextItem) and hasattr(prev_text_item, 'text') and prev_text_item.text:
                            words = prev_text_item.text.split()
                            # Lấy n từ cuối cùng làm ngữ cảnh
                            previous_text_snippet = " ".join(
                                words[-n_tokens_previous:])

                # --- Xử lý DataFrame của bảng ---
                df = preprocess_df(table_item.export_to_dataframe())
                if df.empty:
                    continue
                df_string = get_input_df(df, n_rows=7)
                column_types = get_column_types(df=df)
                n_columns = len(df.columns)

                # --- Xây dựng prompt ---
                prompt_part = ""
                if previous_text_snippet:
                    prompt_part += f"Context before table:\n{previous_text_snippet}\n\n"
                prompt_part += f"Table {table_index}:\n{df_string}\nNumber of columns: {n_columns}\nColumn types: {column_types}\n\n"

                full_prompt += prompt_part
                tables.append(df)
                table_index += 1

    print(f"{len(tables)} tables extracted and processed with preceding context.")
    return tables, full_prompt

### Hàm call LLM API

In [7]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import json
import os

from google import genai
from google.genai import types

# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY_1')
# from dotenv import load_dotenv
# load_dotenv()

# KAGGLE KEY
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")


def generate(prompt):
    model = "gemini-2.0-flash-lite"
    client = genai.Client(
        api_key=GEMINI_API_KEY,
    )
    system_prompt = """You are an AI assistant specialized in analyzing and grouping data tables for vertical concatenation. I will provide you with information for multiple tables. Each table entry might include:

Context before table: A line indicating the subject matter preceding the table. This context can provide clues about the table's topic.
Table index: (e.g., Table 0).
Sample data: A few lines of data. Values in data lines are separated by the = character. The first line of sample data may or may not be an explicit header row containing column names.
Number of columns.
A list of data types for the columns.

The tables will be presented sequentially.

Your tasks are:

First, for each table, meticulously examine its first line of sample data. Determine if this first line explicitly serves as a header row containing distinct column names, rather than being a row of actual data. Identify the table indexes of only those tables that possess such an explicit, pre-existing header row in their sample data. If a table's first row contains data values, even if structured, it should not be considered as having an explicit pre-existing header in this step. It is possible that no tables, or only some, have explicit pre-existing headers.

Second, identify groups of tables that can be vertically concatenated. Tables are considered concatenable if they share the following characteristics:

Relevant Context: The Context before table information, if provided, suggests the tables belong to the same logical section or topic. Use this context as a strong indicator for grouping.
Same Number of Columns.
Compatible Data Types for corresponding columns.
Consistent Header Structure (either pre-existing or to be inferred): If a table in a potential group has an explicit pre-existing header (as identified in the first task), other tables considered for this group should ideally align with this structure. For tables without explicit pre-existing headers, their structure must be compatible with the anticipated common header structure for the group.
Third, for every table, you must provide a definitive list of column headers. Also, for each table, you must determine if these provided headers were pre-existing in the sample data or if they were inferred/predicted by you.
If a table was identified in the first task as having an explicit pre-existing header, you must use those exact pre-existing headers for that table. The determination of whether these headers were guessed or not should reflect this initial finding (i.e., not guessed).
If a table was not identified in the first task as having an explicit pre-existing header, you are required to infer or predict appropriate column headers for it. When inferring headers for a table that is part of a concatenation group, prioritize using or adapting the explicit pre-existing headers from another table within the same group, if such a table exists. If the table is independent or no table in its group has an explicit pre-existing header, infer the headers based on its data content, data types, and any available context. For any table where headers are inferred or predicted by you, this fact must be indicated.

Finally, when constructing the information about headers for all tables, pay attention to consolidation. If multiple table indexes, particularly those grouped together for concatenation as identified in the second task, share the exact same list of column headers and the same status regarding whether those headers were guessed or not, you should consolidate them into a single descriptive entry. This single entry should list all relevant table indexes in its table_index array. For tables that do not fit into such a consolidated group (e.g., independent tables with unique headers, or groups with varying header statuses), create individual descriptive entries as appropriate.

Group the tables based on these criteria for concatenation.

Below is the table data:"""
    content = system_prompt + prompt
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        max_output_tokens=1000,
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
            type=genai.types.Type.OBJECT,
            required=["concatable_tables", "has_headers", "headers_info"],
            properties={
                "concatable_tables": genai.types.Schema(
                    type=genai.types.Type.ARRAY,
                    items=genai.types.Schema(
                        type=genai.types.Type.OBJECT,
                        required=["table_index"],
                        properties={
                            "table_index": genai.types.Schema(
                                type=genai.types.Type.ARRAY,
                                items=genai.types.Schema(
                                    type=genai.types.Type.NUMBER,
                                ),
                            ),
                        },
                    ),
                ),
                "has_headers": genai.types.Schema(
                    type=genai.types.Type.ARRAY,
                    items=genai.types.Schema(
                        type=genai.types.Type.NUMBER,
                    ),
                ),
                "headers_info": genai.types.Schema(
                    type=genai.types.Type.ARRAY,
                    items=genai.types.Schema(
                        type=genai.types.Type.OBJECT,
                        required=["table_index", "headers",
                                  "is_header_guessed"],
                        properties={
                            "table_index": genai.types.Schema(
                                type=genai.types.Type.ARRAY,
                                items=genai.types.Schema(
                                    type=genai.types.Type.NUMBER,
                                ),
                            ),
                            "headers": genai.types.Schema(
                                type=genai.types.Type.ARRAY,
                                items=genai.types.Schema(
                                    type=genai.types.Type.STRING,
                                ),
                            ),
                            "is_header_guessed": genai.types.Schema(
                                type=genai.types.Type.BOOLEAN,
                            ),
                        },
                    ),
                ),
            },
        ),
        system_instruction=[
            types.Part.from_text(text=system_prompt),
        ],
    )
    input_tokens = client.models.count_tokens(model=model, contents=[content])
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    output_tokens = client.models.count_tokens(
        model=model, contents=response.text)

    json_response = json.loads(response.text)
    json_response.update({"input_tokens": input_tokens,
                         "output_tokens": output_tokens})
    return json_response

Ghép nối các nhóm tables lại với nhau

In [8]:
def main_concatenation_logic(
    tables: list[pd.DataFrame], concat_json: dict
) -> list[pd.DataFrame]:
    """
    Concatenate DataFrames based on groups in concat_json.
    Headers are applied based on 'headers_info' from concat_json.
    'is_header_guessed' from 'headers_info' determines if solve_non_header_table is used.
    """
    concatenated_results = []
    # Keep track of indices handled by 'concatable_tables'
    processed_indices_from_json = set()

    if not isinstance(tables, list) or not all(
        isinstance(t, pd.DataFrame) for t in tables
    ):
        print("Error: 'tables' must be a list of DataFrames.")
        return []

    # 1. Pre-process headers_info into a usable map
    # header_details_map: key is table_index (int), value is {'headers': list[str], 'is_guessed': bool}
    header_details_map = {}
    if isinstance(concat_json, dict):
        headers_info_list = concat_json.get("headers_info", [])
        if isinstance(headers_info_list, list):
            for info_entry in headers_info_list:
                if isinstance(info_entry, dict) and \
                   "table_index" in info_entry and \
                   "headers" in info_entry:

                    table_indices_for_this_info = info_entry["table_index"]
                    headers_for_this_info = info_entry["headers"]
                    # Default is_header_guessed to True if missing, common for LLM-inferred headers
                    is_guessed_for_this_info = info_entry.get(
                        "is_header_guessed", True)

                    # Ensure table_indices_for_this_info is a list for iteration
                    if not isinstance(table_indices_for_this_info, list):
                        if isinstance(table_indices_for_this_info, int):
                            # Convert single int to list
                            table_indices_for_this_info = [
                                table_indices_for_this_info]
                        else:
                            print(
                                f"Warning: 'table_index' in headers_info is not a list or int: {info_entry}. Skipping entry.")
                            continue

                    if isinstance(headers_for_this_info, list):
                        for idx in table_indices_for_this_info:
                            if isinstance(idx, int):
                                # First-come, first-served for an index to prevent overriding by less specific entries
                                if idx not in header_details_map:
                                    header_details_map[idx] = {
                                        "headers": headers_for_this_info,
                                        "is_guessed": is_guessed_for_this_info
                                    }
                            else:
                                print(
                                    f"Warning: Non-integer index '{idx}' found in headers_info.table_index list. Skipping this index.")
                    else:
                        print(
                            f"Warning: 'headers' in headers_info entry is not a list: {info_entry}. Skipping entry.")
                else:
                    print(
                        f"Warning: Invalid headers_info entry (e.g., not a dict, or missing keys): {info_entry}. Skipping entry.")
        elif headers_info_list is not None:  # Present but not a list
            print("Warning: 'headers_info' in concat_json is not a list. No explicit header information will be used from headers_info.")
    # If concat_json is not a dict, or headers_info is None/missing, map remains empty.

    if not header_details_map:
        print("Info: 'header_details_map' is empty after processing 'headers_info'. "
              "Tables will be processed using original columns or potentially skipped if in groups.")

    # 2. Process concatable_tables (groups)
    instructions_list = []
    if isinstance(concat_json, dict):
        instructions_list = concat_json.get("concatable_tables", [])
        if not isinstance(instructions_list, list):
            print("Warning: 'concatable_tables' in concat_json is not a list. No concatenation will be performed based on it.")
            instructions_list = []
    elif concat_json is not None:  # concat_json exists but is not a dict
        print("Warning: 'concat_json' is not a dictionary. No concatenation will be performed based on it.")

    for group_instruction in instructions_list:
        if not isinstance(group_instruction, dict) or "table_index" not in group_instruction:
            print(
                f"Warning: Invalid group instruction: {group_instruction}. Skipping this group.")
            continue

        table_indices_from_json_group = group_instruction.get(
            "table_index", [])
        if not isinstance(table_indices_from_json_group, list) or not table_indices_from_json_group:
            print(
                f"Warning: 'table_index' in group is invalid or empty: {group_instruction}. Skipping this group.")
            continue

        valid_indices_in_this_json_group = []
        for index in table_indices_from_json_group:
            if not isinstance(index, int):
                print(
                    f"Warning: Non-integer index '{index}' in group {table_indices_from_json_group}. Skipping this index.")
                continue
            if 0 <= index < len(tables):
                valid_indices_in_this_json_group.append(index)
            else:
                print(
                    f"Warning: Index {index} out of range for tables list. Skipping this index in group {table_indices_from_json_group}.")

        if not valid_indices_in_this_json_group:
            print(
                f"Warning: No valid DataFrames to process for index group {table_indices_from_json_group} after validation.")
            continue

        # Mark all valid indices from this group instruction as "processed" by a concat rule
        for idx in valid_indices_in_this_json_group:
            processed_indices_from_json.add(idx)

        # Determine the header standard for this group
        group_level_header_details = None
        reference_idx_for_group_header = -1

        # Try to find header_info for any table in the group to act as the standard
        for idx_in_group in valid_indices_in_this_json_group:
            if idx_in_group in header_details_map:
                group_level_header_details = header_details_map[idx_in_group]
                reference_idx_for_group_header = idx_in_group
                # print(f"Info: Using header_info from table {reference_idx_for_group_header} as standard for group {valid_indices_in_this_json_group}.")
                break  # Found a standard for the group

        if group_level_header_details is None:
            print(f"Critical Warning: No 'headers_info' found for ANY table in group {valid_indices_in_this_json_group}. "
                  "These tables will be added individually if not processed otherwise.")
            # Add original tables to results to prevent data loss, they won't be concatenated as a group
            for original_df_idx in valid_indices_in_this_json_group:
                # Check if already added (e.g. by another rule or if logic changes)
                # This check isn't strictly necessary with current flow but good for thought
                # if original_df_idx not in [item.attrs.get('original_index', -1) for item in concatenated_results]: # pseudo-code for check
                concatenated_results.append(
                    tables[original_df_idx].copy())  # Add original
            continue  # Skip to next group instruction

        group_target_headers = group_level_header_details["headers"]
        is_group_header_guessed = group_level_header_details["is_guessed"]

        dfs_to_concat_in_group = []
        for original_df_idx in valid_indices_in_this_json_group:
            df = tables[original_df_idx].copy()

            # All tables in this group will use the same target headers and guessed status
            # derived from the group_level_header_details.

            if len(df.columns) != len(group_target_headers):
                print(f"Warning: Table at index {original_df_idx} (in group) has {len(df.columns)} columns, "
                      f"but group standard header has {len(group_target_headers)} columns. Skipping this table from group.")
                # Optionally, add this skipped table as an independent one later if robust handling is needed
                # For now, it's skipped from this group.
                continue

            if is_group_header_guessed:
                df = solve_non_header_table(df, group_target_headers)
            else:
                df.columns = group_target_headers

            dfs_to_concat_in_group.append(df)

        if dfs_to_concat_in_group:
            if len(dfs_to_concat_in_group) == 1:  # Only one table qualified
                concatenated_results.append(dfs_to_concat_in_group[0])
            else:
                try:
                    final_df_group = pd.concat(
                        dfs_to_concat_in_group, ignore_index=True)
                    concatenated_results.append(final_df_group)
                except Exception as e:
                    print(
                        f"Error concatenating DataFrames in group {valid_indices_in_this_json_group}: {e}. Adding DFs individually.")
                    # Add prepared DFs individually
                    concatenated_results.extend(dfs_to_concat_in_group)
        # Group had valid indices but no DFs made it (e.g., all column mismatches)
        elif valid_indices_in_this_json_group:
            print(
                f"Warning: No DataFrames in group {valid_indices_in_this_json_group} could be prepared for concatenation.")

    # 3. Handle independent tables (those not covered by any 'concatable_tables' group instruction)
    all_available_indices = set(range(len(tables)))
    independent_table_indices = sorted(
        list(all_available_indices - processed_indices_from_json))

    for idx in independent_table_indices:
        # Keep a reference to original
        original_df_for_independent = tables[idx]
        df = original_df_for_independent.copy()

        header_details_for_independent_df = header_details_map.get(idx)

        if header_details_for_independent_df is None:
            print(
                f"Info: No 'headers_info' found for independent table at index {idx}. Adding table as is.")
            # Add with original/current headers
            concatenated_results.append(df)
            continue

        target_headers = header_details_for_independent_df["headers"]
        is_guessed = header_details_for_independent_df["is_guessed"]

        if len(df.columns) != len(target_headers):
            print(f"Warning: Column count mismatch for independent table {idx}. Original has {len(df.columns)}, "
                  f"target has {len(target_headers)}. Adding table as is (original form).")
            concatenated_results.append(
                original_df_for_independent.copy())  # Add original on error
            continue

        if is_guessed:
            df = solve_non_header_table(df, target_headers)
        else:
            df.columns = target_headers

        concatenated_results.append(df)

    return concatenated_results

### Kết quả

## 4. Full pipeline

In [9]:
from IPython.display import Markdown, display


def get_tables_from_sources(
    sources: list[str], output_path: str = None
) -> dict[str, list[pd.DataFrame]]:
    processed_tables = {}
    conversions = convert_pdf_to_docling(sources)
    total_tokens = {"input_tokens": [], "output_tokens": []}
    for source in sources:
        pdf_name = source.split("/")[-1][:-4]
        print(f"Processing {pdf_name}")
        try:
            tables, full_prompt = get_tables_from_docling(conversions[source])
            json_instructions = generate(full_prompt)

            source_input_tokens = json_instructions.get("input_tokens")
            source_output_tokens = json_instructions.get("output_tokens")

            # print(f"{pdf_name} - input_tokens = {source_input_tokens}")
            # print(f"{pdf_name} - output_tokens = {source_output_tokens}")
            print(
                f"{pdf_name} - full_json = {json_instructions}"
            )

            total_tokens["input_tokens"].append(source_input_tokens)
            total_tokens["output_tokens"].append(source_output_tokens)

            concatenated_dataframes = main_concatenation_logic(
                tables, json_instructions
            )
            processed_tables[source] = concatenated_dataframes
            if output_path:
                for i, df in enumerate(concatenated_dataframes):
                    df.to_csv(f"{pdf_name}_{i}.csv", index=False)
                    print(f"\n--- Concatenated DataFrame Group {i + 1} ---")
                    display(Markdown(df.to_markdown()))
                    print(f"Columns: {df.columns.tolist()}")
                    print(f"Shape: {df.shape}")
            print(f"Processed {pdf_name}")
            print("=" * 50)
        except Exception as e:
            print(f"Error processing {source}: {e}")
    return processed_tables, total_tokens

In [10]:
# SETTING INPUT AND OUTPUT PATH
import os

input_path = "/kaggle/input/tables-pdf"

sources = [
    os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(".pdf")
]
print("Source:", sources)

pdf_names = [pdf_path.split("/")[-1] for pdf_path in sources]
print("\n\n")
output_path = "/kaggle/working/"
print("Output path:", output_path)

Source: ['/kaggle/input/tables-pdf/c935e2902adf7040a6ffe0db0f7c11e6.pdf', '/kaggle/input/tables-pdf/national-capitals.pdf', '/kaggle/input/tables-pdf/CA Warn Report.pdf', '/kaggle/input/tables-pdf/659dfd3e5c22a64d7597cb50c6fc455e.pdf', '/kaggle/input/tables-pdf/0e6af4a364ff3fe835bd4e332a2fb9cf.pdf', '/kaggle/input/tables-pdf/f8a04d6beeed137eeb7840a5c459842d.pdf', '/kaggle/input/tables-pdf/78f480a26ad6a8c7c064e9d9fd5b2c4f.pdf']



Output path: /kaggle/working/


In [11]:
processed_tables, total_tokens = get_tables_from_sources(
    sources, output_path=output_path
)

Time taken: 184.5845
Processing c935e2902adf7040a6ffe0db0f7c11e6
10 tables extracted and processed with preceding context.
c935e2902adf7040a6ffe0db0f7c11e6 - full_json = {'concatable_tables': [{'table_index': [0, 1, 2, 3, 4, 5, 6, 7]}], 'has_headers': [], 'headers_info': [{'table_index': [0, 1, 2, 3, 4, 5, 6, 7], 'headers': ['Air Jordan', 'Year', 'Description'], 'is_header_guessed': True}, {'table_index': [8], 'headers': ['V · T · E', 'Nike, Inc.', 'Links to related articles'], 'is_header_guessed': True}, {'table_index': [9], 'headers': ['Categories: Sportswear brands', 'Categories: Sportswear brands', 'Nike brands', 'Michael Jordan', 'Products introduced in 1984', 'Products introduced in 1984'], 'is_header_guessed': True}], 'input_tokens': CountTokensResponse(total_tokens=2193, cached_content_token_count=None), 'output_tokens': CountTokensResponse(total_tokens=322, cached_content_token_count=None)}

--- Concatenated DataFrame Group 1 ---


|    | Air Jordan         | Year   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|---:|:-------------------|:-------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Air Jordan IV      | 1989   | In December 1988, Nike released the Air Jordan IV to the public. Designed by Tinker Hatfield, it was the first Air Jordan released on the global market. It had four colorways: White/Black, Black/Cement Grey, White/Fire Red-Black, and Off White/Military Blue. Nike featured director and actor Spike Lee in ads for the shoe. [27]  Lee had featured the shoe in his movie  Do The Right Thing . [25] Michael Jordan wore the Air Jordan IV when he made "The Shot", a series winner in Game 5 of the 1989 NBA First Round between the Chicago Bulls and the Cleveland Cavaliers. In 2012 a Cavalier colorway dubbed the "Cavs" was released to honor "The Shot". |
|  1 | Air Jordan V       | 1990   | The Air Jordan V was released in February 1990 and designed by Hatfield. Inspired by a WWII Mustang fighter, [25] features include a reflective tongue with a protruding design, translucent rubber soles and lace locks. [28] The Air Jordan V saw a lot of use in popular sitcom  The Fresh Prince of Bel-Air . During many episodes Will Smith wore the Metallic Silver, Grape, and Fire Red colorways. To pay tribute to his character, Jordan released the Air Jordan 5 Bel Air in 2013 and 2020. [29][30]                                                                                                                                                        |
|  2 | Air Jordan VI      | 1991   | Designed after a German sports car, Michael Jordan wore the VI for his first Bulls championship. [25]  The shoes were worn by the protagonist of the manga  Slam Dunk , Hanamichi Sakuragi. In 2014, Nike released special versions of the VI featuring artwork from the series. [31]  Various other models in the Air Jordan line are featured in the series, including the original Air Jordan, the V, and the XII. [32]                                                                                                                                                                                                                                             |
|  3 | Air Jordan VII     | 1992   | The Air Jordan VII introduced "huarache" technology which allowed the shoes to better conform to the user's foot. A few things were no longer featured on the new model, such as the visible air sole, the Nike Air logo, and the translucent soles. This was the first Air Jordan in the line that did not have any distinctive "Nike Air" branding on the outer portions of the shoe, only on the in-soles. When Jordan went to compete at the 1992 Summer Olympics to play for the US Men's Basketball Team (also known as the "Dream Team"), Nike released a special                                                                                               |
|  4 |                    |        | Olympic color combo of the Air Jordan VII model which had Jordan's Olympic jersey number 9, instead of the usual "23" found on other colorways. [25] Various models of the Air Jordan VII were re-released beginning with its 10-year anniversary in 2002. [33]                                                                                                                                                                                                                                                                                                                                                                                                        |
|  5 | Air Jordan VIII    | 1993   | The Air Jordan VIII was released to coincide with the 1992- 93 NBA season. The eighth model of the Air Jordan contains a full-length air sole, polyurethane midsole, polycarbonate shank plate, and two crossover straps. The VIIIs were known for a successful ad campaign in which Bugs Bunny appeared alongside Michael Jordan to market the shoes. [25]  The shoe was re-released in 2003, 2007, 2008, 2013, and 2015-2017.                                                                                                                                                                                                                                        |
|  6 | Air Jordan IX      | 1993   | Originally released in November 1993, the Air Jordan IX was the first model released after Michael Jordan's retirement. Jordan never played an NBA season wearing these shoes. This model was inspired by baseball cleats that Jordan wore when playing minor-league baseball. [25]  The shoe was re- released in 2002, 2008, 2010, 2012, and 2014-2018. Like the VII and VIII models, the Air Jordan IX featured an inner sock sleeve and nubuck accents. The sole featured different symbols and languages of different countries. The Air Jordan IX is depicted to be worn by the Statue of Michael Jordan inside the United Center in Chicago. [34]                |
|  7 | Air Jordan X       | 1994   | This was released in different colors representing U.S. cities. It was the first Air Jordan to feature a lightweight Phylon midsole. The shoe also featured all of Michael Jordan's accomplishments up to his first retirement on the outsole. [25] The shoe was re-released in 2005, 2008, 2012-2016, and 2018.                                                                                                                                                                                                                                                                                                                                                       |
|  8 | Air Jordan XI      | 1995   | This model was designed by Tinker Hatfield. When the shoe launched, Michael Jordan (retired from basketball by then) was with the Birmingham Barons in baseball's minor baseball leagues. Hatfield designed the sneaker waiting for Jordan to come back and hoping he would play in them. [35] The ballistic mesh upper of the sneaker was meant to make the Air Jordan XI lighter and more durable. Further changes                                                                                                                                                                                                                                                   |
|  9 |                    | 1996   | Michael Jordan's favorite. [39].The Air Jordan XII featured a "Rising Sun" motif mimicking the Flag of Japan. [25]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 10 |                    | 1997   | This model had a carbon fiber plate, designed by Hatfield. The black panther was the inspiration for the Air Jordan XIII, with the sole resembling the pads on a panther's paw. A hologram on the back of the shoe imitates a panther's eyes in the dark. [25]  They were re-released in 2005, which coincided with the release of the Air Jordan 8s shoe. In the movie  He Got Game , director Spike Lee had access to the Air Jordan XIII months before it was available to the public or even worn by Jordan himself and featured it in scenes. [37]                                                                                                                |
| 11 | Air Jordan XIV     | 1998   | Inspired by the Ferrari 550 M which Michael Jordan owned, the Air Jordan XIV was originally released in October 1998. [25]  It was re-released in 2005, 2006, 2008, 2011, 2012 and 2014-2018. The Air Jordan XIV co-styled by Hatfield and Mark Smith was made race ready and equipped with the Jordan Jumpman insignia on a Ferrari shaped badge. The color scheme of predominant black accentuated with red was nicknamed "The Last Shot" because Michael Jordan wore them as he hit the game winning shot over Bryon Russell, of the Utah Jazz,                                                                                                                     |
| 12 |                    |        | in his final game with the Chicago Bulls in the 1998 NBA Finals. There are 14 Jumpman logos-7 on each shoe- corresponding the shoes' number in the series.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 13 | Air Jordan XV      | 1999   | This was the first shoe after Jordan's retirement. The design was inspired by the North American X-15, which was developed by NASA during the 1950s. The sides of the XV were made from woven kevlar fiber. [25]  The Jordan XVs were Hatfield's least favorite in the series. [38]                                                                                                                                                                                                                                                                                                                                                                                    |
| 14 | Air Jordan XVI     | 2001   | The shoe came with spats, and the design was inspired by performance cars and architecture. The ad campaign featured Mos Def. [25]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 15 | Air Jordan XVII    | 2002   | This pair of Jordans came with a multimedia CD-ROM containing the Air Jordan XVII song. The retail price of the shoe was US$200. [25]  The defining functional design element of the Air Jordan XVII model, which was later replicated on the Air Jordan XXIII model, was the reinforced mid-sole which provided a sturdy and stable chassis for the shoe. They were made in four mid top colors and three low-top colors. Michael Jordan wore the XVII while playing for the Washington Wizards, after his second retirement return. [41] The shoe was re-released in 2008 and 2016.                                                                                  |
| 16 | Air Jordan XVIII   | 2003   | The Air Jordan XVIII shoe was released during Michael Jordan's last season, in which he played for the Washington Wizards. The shoe was designed by Air Jordan Senior Footwear Designer, Tate Kuerbis, who had been part of the Jordan footwear design team since 1999 and with Nike since 1995. The inspiration for the design came from the carbon fiber- based monocoque of F1 race cars, [25]  race car driving shoes (rubber heel wrap) and Fine Italian dress shoes (bold stitching on the soles). It was re-released in 2008.                                                                                                                                   |
| 17 | Air Jordan XXI     | 2006   | The Air Jordan XXI model of shoes was designed by D'Wayne Edwards and inspired by sport touring vehicles. The shoe features lower-foot air grilles, double-overlasted Phylon midsole, a carbon fiber shank plate and a seamless diamond-quilted bootie. It came with removable parts that could make the cushioning firm or soft, and had text that could be seen under a blacklight. [25] The Air Jordan XXI was introduced on television by the "Second Generation" advertisement.                                                                                                                                                                                   |
| 18 |                    | 2007   | The XX2 was inspired by the F-22 Raptor. [25]  The promo commercial was directed by Mark Romanek. [44]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 19 | Air Jordan XX3     | 2008   | The Air Jordan XX3 was designed by Tinker Hatfield. It was the first basketball shoe in the "Nike Considered" category, for using materials from not more than 200 miles (320 km) from a Nike Factory. It features a hand-stitched exterior, full- length bootie, carbon fiber shank plate, the last to feature interchangeable IPS pillars, and an articulated chassis. The shoe was released on January 25, 2008, and was the last Air Jordan until the XX8 to have Roman numeral identification. The shoe was re-released in 2015-2016.                                                                                                                             |
| 20 | Air Jordan 2009    | 2009   | The Air Jordan 2009 was designed by Jason Mayden and was the first Air Jordan model named after the year of its release rather than its numbered system. Inspired by Jordan's defensive focus, the shoe incorporates Articulated Propulsion Technology used by Paralympian runners. It also features a durable pleated silk upper, protective Thermoplastic polyurethane (TPU) chassis, carbon fiber arch plate and Zoom Air structure. The shoe was released on January 31, 2009, and has not been re-released.                                                                                                                                                       |
| 21 | Air Jordan 2010    | 2010   | This was released during the 25th anniversary of the Air Jordan brand. The base of each midsole has stylized text that when combined reads: "I've failed over and over and over again in my life. And that is why I succeed." This quote is a reference to a 1997 advertising campaign, with Michael Jordan detailing his failures that led to his career successes.                                                                                                                                                                                                                                                                                                   |
| 22 | Air Jordan 2011    | 2011   | The shoe has interchangeable insoles [25]  - a red one for power and a blue one for quickness. Four colorways of the shoe were released corresponding with the 2011 All Star Game: White/Black, White/Red and White/Blue that represented the East/West Jersey Colors. The "Year of the Rabbit" colorway was a limited release that celebrated Michael Jordan's Chinese zodiac sign. The 2011 has a star-constellation pattern that also serves as ventilation. It uses patent leather wrapped around the shoe. The shoes are hand burnished and crafted. A dress shoe that feels similar to the XI was reportedly the goal. The shoe has not been re-released.        |
| 23 | Air Jordan 2012    | 2012   | The Air Jordan 2012 offers six customization configurations. Two interchangeable sleeves and three insoles adapt to different playing styles. The Deluxe model was launched on February 8, while the customization Flight models were released on February 25, 2012. It was the final Air Jordan model to be named after the year it was released as the numbered system returned in 2013 with the Air Jordan XX8.                                                                                                                                                                                                                                                     |
| 24 | Air Jordan XX8     | 2013   | The Air Jordan XX8, designed by Tinker Hatfield, was released on February 16, 2013. The outside shroud was made from a Swiss fabric used for motorcycle jackets. [25]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 25 | Air Jordan XX9     | 2014   | The Air Jordan XX9, also designed by Hatfield, released in September 2014 in both an elephant print and a knit edition. The shoe debuted in the NBA by Russell Westbrook and Kawhi Leonard. The shoe has a performance woven upper, with areas that are stiff and others that are more flexible. [25] Jordan released two versions, a regular cut and low version. Some of the colorways released in low version are buckets, Chicago bulls, UNC, and infrared.                                                                                                                                                                                                        |
| 26 | Air Jordan XXX     | 2016   | The Air Jordan XXX was again designed by Tinker Hatfield. The first colorway of the shoe released on February 16. [45] The shoe consists of an upper and outsole similar to the XX9. The upper has a flyknit-constructed ankle collar that overextends slightly. The outsole has a more noticeable change, with a different traction pattern, while the midsole remains almost identical.                                                                                                                                                                                                                                                                              |
| 27 | Air Jordan XXXI    | 2016   | The Air Jordan XXXI is heavily influenced by the Air Jordan 1s, having a leather upper and swoosh, Jumpman, and Jordan "Wings" logos. Its retail debut was on September 3, 2016, in the "Banned" colorway, for the 30th anniversary of the NBA banning the Air Jordan 1. [25][46]  Notable appearances of the shoe include the "USA" colorway worn during the 2016 Olympic basketball tournament by members of team USA. [47]                                                                                                                                                                                                                                          |
| 28 | Air Jordan XXXII   | 2017   | The Air Jordan XXXII was influenced by the Air Jordan 2 and included a Jordan "Wings" logo. It first released in the "Rossa Corsa" colorway on September 23, 2017. A "Banned" colorway was released on October 18, 2017. Another special edition called the "Russ" colorway was released to celebrate Russell Westbrook's sponsorship with Jordan Brand. Jordan Brand released 2 types of this shoe, the original mid length cut and low cut. [25]                                                                                                                                                                                                                     |
| 29 | Air Jordan XXXIII  | 2018   | The Air Jordan XXXIII was released on October 18, 2018. This is the first Air Jordan model to go laceless. [48]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 30 | Air Jordan XXXIV   | 2019   | The Air Jordan XXXIV was released on September 25, 2019, with details such as a commemorative date for the brand's 30th anniversary. [49]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 31 | Air Jordan XXXV    | 2020   | The Air Jordan XXXV debuted in the Fall of 2020 with a distinct new shape. The model focused on weight reduction for responsiveness. [50]  An area of the midfoot had a hole to provide stability, bounce, and comfort. [51]  The Air Jordan had several collaborations with contemporary NBA players, including Jayson Tatum and Zion Williamson.                                                                                                                                                                                                                                                                                                                     |
| 32 |                    | 2021   | The Air Jordan XXXVI was first teased by German-American professional basketball player Satou Sabally via social media, and made its on-court debut on March 25, 2021. [52]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 33 | Air Jordan XXXVII  | 2022   | The Air Jordan XXXVII was released on July 28, 2022. [53]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 34 | Air Jordan XXXVIII | 2023   | The Air Jordan XXXVIII contains a Cushlon 3.0 midsole along with a Zoom Strobel Unit. It was released on August                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 35 |                    |        | 18, 2023, for $200.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 36 | Air Jordan XXXIX   | 2024   | The Air Jordan XXXIX was released in July 2024.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |

Columns: ['Air Jordan', 'Year', 'Description']
Shape: (37, 3)

--- Concatenated DataFrame Group 2 ---


|    | V · T · E       | Nike, Inc.                | Links to related articles   |
|---:|:----------------|:--------------------------|:----------------------------|
|  0 | V   ·  T   ·  E | Nike, Inc.                | [show]                      |
|  1 |                 | Links to related articles | [show]                      |

Columns: ['V · T · E', 'Nike, Inc.', 'Links to related articles']
Shape: (2, 3)

--- Concatenated DataFrame Group 3 ---


...

Columns: ['Years', 'Sponsor', 'Bonuses']
Shape: (2, 3)

--- Concatenated DataFrame Group 5 ---


|    | †   | *                                                                                  |
|---:|:----|:-----------------------------------------------------------------------------------|
|  0 | *   | Denotes winners of the Derby and Preakness but not the Belmont                     |
|  1 | #   | Denotes winners of the other two combinations of 2 out of the 3 Triple Crown races |

Columns: ['†', '*']
Shape: (2, 2)

--- Concatenated DataFrame Group 6 ---


|    |   Year | Kentucky Derby   | Preakness Stakes   | Belmont Stakes   |
|---:|-------:|:-----------------|:-------------------|:-----------------|
|  0 |   1867 |                  |                    | Ruthless [Fy]    |

Columns: ['Year', 'Kentucky Derby', 'Preakness Stakes', 'Belmont Stakes']
Shape: (1, 4)
Processed 78f480a26ad6a8c7c064e9d9fd5b2c4f


In [12]:
for source in processed_tables:
    file_name = source.split("/")[-1][:-4]
    print(f"{file_name} has {len(processed_tables[source])} tables")

c935e2902adf7040a6ffe0db0f7c11e6 has 3 tables
national-capitals has 1 tables
CA Warn Report has 2 tables
659dfd3e5c22a64d7597cb50c6fc455e has 4 tables
0e6af4a364ff3fe835bd4e332a2fb9cf has 2 tables
f8a04d6beeed137eeb7840a5c459842d has 5 tables
78f480a26ad6a8c7c064e9d9fd5b2c4f has 6 tables


In [13]:
total_input_tokens = sum(
    [count_tokens.total_tokens for count_tokens in total_tokens["input_tokens"]]
)
total_output_tokens = sum(
    [count_tokens.total_tokens for count_tokens in total_tokens["output_tokens"]]
)

In [14]:
api_price = {
    "input_tokens": 0.075,  # per 1million
    "output_tokens": 0.3,
}

print("Total input tokens:", total_input_tokens)
print("Total onput tokens:", total_output_tokens)

print(
    "Price =",
    (total_input_tokens / 1e6) * api_price["input_tokens"]
    + (total_output_tokens / 1e6) * api_price["output_tokens"],
)

Total input tokens: 17304
Total onput tokens: 2393
Price = 0.0020157
